# Importing Packages

In [21]:
import pandas as pd
import numpy as np
import cv2
from skimage import io
from glob import glob
import os
import pydicom
import matplotlib.pyplot as plt
import albumentations as A
from PIL import Image
from skimage.color import label2rgb
import random

In [22]:
df = pd.read_csv("../data/train_df.csv", index_col=0)

In [23]:
df.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target,gen_enc,site_enc,diag_enc
0,ISIC_2637011,IP_7279968,IL_7972535,male,45.0,head/neck,unknown,benign,0,1,0,8
1,ISIC_0015719,IP_3075186,IL_4649854,female,45.0,upper extremity,unknown,benign,0,0,6,8
2,ISIC_0052212,IP_2842074,IL_9087444,female,50.0,lower extremity,nevus,benign,0,0,1,5
3,ISIC_0068279,IP_6890425,IL_4255399,female,45.0,head/neck,unknown,benign,0,0,0,8
4,ISIC_0074268,IP_8723313,IL_6898037,female,55.0,upper extremity,unknown,benign,0,0,6,8


# Assigning Paths

In [24]:
# === DICOM ===
# Create the paths
path_tr_dcm = '../split/train_dcm/' + df['file'] + '.dcm'
path_tr_jpg = '../split/train/mel/2020/' + df['file'] + '.jpg'

# Append to the original dataframes
df['path_dcm'] = path_tr_dcm
df['path_jpg'] = path_tr_jpg

In [25]:
df['dcm'] = df.file.apply(lambda x: str(x) + '.dcm')
df['jpg'] = df.file.apply(lambda x: str(x) + '.jpg')

In [26]:
df.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target,gen_enc,site_enc,diag_enc,path_dcm,path_jpg,dcm,jpg
0,ISIC_2637011,IP_7279968,IL_7972535,male,45.0,head/neck,unknown,benign,0,1,0,8,../split/train_dcm/ISIC_2637011.dcm,../split/train/mel/2020/ISIC_2637011.jpg,ISIC_2637011.dcm,ISIC_2637011.jpg
1,ISIC_0015719,IP_3075186,IL_4649854,female,45.0,upper extremity,unknown,benign,0,0,6,8,../split/train_dcm/ISIC_0015719.dcm,../split/train/mel/2020/ISIC_0015719.jpg,ISIC_0015719.dcm,ISIC_0015719.jpg
2,ISIC_0052212,IP_2842074,IL_9087444,female,50.0,lower extremity,nevus,benign,0,0,1,5,../split/train_dcm/ISIC_0052212.dcm,../split/train/mel/2020/ISIC_0052212.jpg,ISIC_0052212.dcm,ISIC_0052212.jpg
3,ISIC_0068279,IP_6890425,IL_4255399,female,45.0,head/neck,unknown,benign,0,0,0,8,../split/train_dcm/ISIC_0068279.dcm,../split/train/mel/2020/ISIC_0068279.jpg,ISIC_0068279.dcm,ISIC_0068279.jpg
4,ISIC_0074268,IP_8723313,IL_6898037,female,55.0,upper extremity,unknown,benign,0,0,6,8,../split/train_dcm/ISIC_0074268.dcm,../split/train/mel/2020/ISIC_0074268.jpg,ISIC_0074268.dcm,ISIC_0074268.jpg


# Melanoma Images

In [27]:
# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(16,6))
# plt.suptitle('Melanoma Images', fontsize=16)
# for i in range(0, 15):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')
    

<img src="../images/melanoma_images.png">

# Non-Melanoma Images

In [8]:
# paths = glob('../split/train/not_mel/*.jpg')
# fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(16,6))
# plt.suptitle('Non-Melanoma Images', fontsize=16)
# for i in range(0, 15):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/non_melanoma_images.png">

# OpenCV Transforms

## Grayscale

In [9]:
# path = "../split/train_dcm"

# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16, 6))
# plt.suptitle('Grayscale', fontsize=16)
# for i in range(0, 10):
#     data = pydicom.read_file(os.path.join(path, df.dcm[i]))
#     image = data.pixel_array

#     image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#     image = cv2.resize(image, (256,256))

#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/grayscale.png">

## Gaussian Blur

In [10]:
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('Gaussian Blur', fontsize=16)
# for i in range(0, 10):
#     data = pydicom.read_file(os.path.join(path, df.dcm[i]))
#     image = data.pixel_array
#     image = cv2.resize(image, (200,200))
#     image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), 256/10), -4, 128)

#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/gaussian_blur.png">

## Hue Saturation Brightness

In [11]:
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('Hue, Saturation, Brightness', fontsize=16)
# for i in range(0, 10):
#     data = pydicom.read_file(df['path_dcm'][i])
#     image = data.pixel_array
    
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
#     image = cv2.resize(image, (200,200))
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/hue_saturation_brightness.png">

## LUV Colorspace

In [12]:
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('LUV Colorspace', fontsize=16)
# for i in range(0, 10):
#     data = pydicom.read_file(df['path_dcm'][i])
#     image = data.pixel_array
    
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
#     image = cv2.resize(image, (200,200))
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/luv_colorspace.png">

# Albumentations Transformations

## Tranpose

In [13]:
# transform = A.Compose([
#     A.Transpose()
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16, 6))
# plt.suptitle('Transpose', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256, 256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/transpose.png">

## Vertical Flip

In [14]:
# transform = A.Compose([
#     A.VerticalFlip()
# ])
# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16, 6))
# plt.suptitle('VerticalFlip', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256, 256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/vertical_flip.png">

## Horizontal FLip

In [15]:
# transform = A.Compose([
#     A.HorizontalFlip()
# ])
# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('HorizontalFlip', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/horizontal_flip.png">

## Random Brightness

In [16]:
# transform = A.Compose([
#     A.RandomBrightness(limit=0.3)
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('RandomBrightness', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/random_brightness.png">

## Random Contrast

In [17]:
# transform = A.Compose([
#     A.RandomContrast(limit=0.3)
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('RandomContrast', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/random_contrast.png">

## CLAHE

In [18]:
# transform = A.Compose([
#     A.CLAHE(clip_limit=6.0)
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('CLAHE', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/clahe.png">

## Hue Saturation Value

In [19]:
# transform = A.Compose([
#     A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10)
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('HueSaturationValue', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/hue_saturation_value.png">

## Shift Scale Rotate

In [20]:
# transform = A.Compose([
#     A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, border_mode=0)
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('ShiftScaleRotate', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/shift_scale_rotate.png">

## Cutout

In [20]:
# image_size = 128

# transform = A.Compose([
#     A.Cutout(max_h_size=32, max_w_size=32, num_holes=8)
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('Cutout', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/cutout.png">

## Motion Blur

In [21]:
# transform = A.Compose([
#     A.MotionBlur(blur_limit=5)
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('MotionBlur', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/motion_blur.png">

## Median Blur

In [22]:
# transform = A.Compose([
#     A.MedianBlur(blur_limit=7)
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('MedianBlur', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/median_blur.png">

## Gaussian Blur

In [23]:
# transform = A.Compose([
#     A.GaussianBlur(blur_limit=7)
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('GaussianBlur', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/gaussian_blur.png">

## Gauss Noise

In [24]:
# transform = A.Compose([
#     A.GaussNoise(var_limit=(5.0, 30.0))
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('GaussNoise', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/gauss_noise.png">

## Optical Distortion

In [25]:
# transform = A.Compose([
#     A.OpticalDistortion(distort_limit=1.0)
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('OpticalDistortion', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/optical_distortion.png">

## Grid Distortion

In [26]:
# transform = A.Compose([
#     A.GridDistortion(num_steps=5, distort_limit=1.)
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('GridDistortion', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/grid_distortion.png">

## Elastic Transform

In [37]:
# transform = A.Compose([
#     A.ElasticTransform(alpha=3)
# ])

# paths = glob('../split/train/mel/2020/*.jpg')
# fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
# plt.suptitle('ElasticTransform', fontsize=16)
# for i in range(0, 10):   
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (256,256))
#     transformed = transform(image=image)
#     aug_image = transformed['image']
   
#     x = i // 5
#     y = i % 5
#     axes[x, y].imshow(aug_image, cmap=plt.cm.bone)
#     axes[x, y].axis('off')

<img src="../images/elastic_transform.png">

# Creating Transformations

In [28]:
transform = A.Compose([
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightness(limit=0.2, p=0.75),
    A.RandomContrast(limit=0.2, p=0.75),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=0.7),
    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.),
        A.ElasticTransform(alpha=3),
    ], p=0.7),
    A.CLAHE(clip_limit=4.0, p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
    A.Resize(256, 256),
    A.Cutout(max_h_size=int(256 * 0.375), max_w_size=int(256 * 0.375), num_holes=1, p=0.7),
    A.Normalize()
])


/Users/examsherpa/opt/anaconda3/envs/nn-env/lib/python3.6/site-packages/albumentations/augmentations/transforms.py:2611: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


In [29]:
# paths = glob('../split/train/mel/2020/*.jpg')

# for i in range(0, 467):   
#     f, ax = plt.subplots()
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     transformed = transform(image=image)
#     aug_image = transformed['image']
#     plt.imshow(aug_image)
#     plt.savefig("../split/train/mel/2020_T/aug_image_{}.png".format(i))
#     plt.close('all')

In [35]:
# paths = glob('../split/train/mel/2019/*.jpg')

# for i in tqdm(range(0, 4522)):   
#     f, ax = plt.subplots()
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     transformed = transform(image=image)
#     aug_image = transformed['image']
#     plt.imshow(aug_image)
#     plt.savefig("../split/train/mel/2019_T/aug_image_{}.png".format(i))
#     plt.close('all')

In [36]:
# paths = glob('../split/train/mel/2018/*.jpg')

# for i in range(0, 1113):   
#     f, ax = plt.subplots()
#     image = cv2.imread(paths[i], cv2.IMREAD_COLOR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     transformed = transform(image=image)
#     aug_image = transformed['image']
#     plt.imshow(aug_image)
#     plt.savefig("../split/train/mel/2018_T/aug_image_{}.png".format(i))
#     plt.close('all')

In [ ]:
def hair_remove(image):
    grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    kernel = cv2.getStructuringElement(1,(17,17))
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    _,threshold = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)
    final_image = cv2.inpaint(image,threshold,1,cv2.INPAINT_TELEA)
    return final_image

In [ ]:
hairy_ = df[df["sex"] == 1].reset_index().iloc[[12, 14, 17, 22, 33, 34]]
image_list = hairy['path_jpg']
image_list = image_list.reset_index()['path_jpg']